In [46]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("MyApp").getOrCreate()

csvQuartosReservados = "/Users/joaobraganca/Documents/ESTG/TEAD/MEI_TP_TEAD/Analise_Dados/DatasetOriginais/Quartos_Reservados.csv"
QuartosReservadosTable = "QuartosReservados"

df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("delimiter", ";")
    .option("trim", "true")
    .load(csvQuartosReservados)
    .withColumnRenamed("Hotel ID", "hotel_ID")
    .withColumnRenamed("Reserve ID", "Reserve_ID")
    .withColumnRenamed("País", "pais")
    .withColumnRenamed("Estado da reserva", "estado_reserva")
    .withColumnRenamed("Room ID", "room_ID")
    .withColumnRenamed("Tipo de Quarto", "tipo_quarto")
    .withColumnRenamed("RatePlan", "rate_plan")
    .withColumn("data_reserva", to_date(col("Data da reserva"), "dd/MM/yyyy"))
    .withColumn("data_chegada", to_date(col("Data chegada"), "dd/MM/yyyy"))
    .drop("Data chegada")
    .withColumn("data_partida", to_date(col("Data de partida"), "dd/MM/yyyy"))
    .drop("Data de partida")
    .withColumnRenamed("Número de noites", "num_noites")
    .withColumnRenamed("Ocupação", "ocupacao")
    .withColumnRenamed("Adultos", "adultos")
    .withColumnRenamed("Crianças", "criancas")
    .withColumnRenamed("Bebés", "bebes")
    .withColumnRenamed("Preço (€)", "preco_euros")
)

df.createOrReplaceTempView(QuartosReservadosTable)
df.cache()
df.printSchema()
df.show()

root
 |-- hotel_ID: integer (nullable = true)
 |-- Reserve_ID: integer (nullable = true)
 |-- pais: string (nullable = true)
 |-- estado_reserva: string (nullable = true)
 |-- room_ID: integer (nullable = true)
 |-- tipo_quarto: string (nullable = true)
 |-- rate_plan: string (nullable = true)
 |-- Data da reserva: timestamp (nullable = true)
 |-- num_noites: integer (nullable = true)
 |-- ocupacao: integer (nullable = true)
 |-- adultos: integer (nullable = true)
 |-- criancas: integer (nullable = true)
 |-- bebes: integer (nullable = true)
 |-- preco_euros: double (nullable = true)
 |-- data_reserva: date (nullable = true)
 |-- data_chegada: date (nullable = true)
 |-- data_partida: date (nullable = true)

+--------+----------+--------------+--------------+-------+--------------------+--------------------+--------------------+----------+--------+-------+--------+-----+-----------+------------+------------+------------+
|hotel_ID|Reserve_ID|          pais|estado_reserva|room_ID|      

23/05/03 21:26:42 WARN CacheManager: Asked to cache already cached data.


In [47]:
# Calcular diferença de dias
df = df.withColumn("days_diff", datediff("data_partida", "data_chegada"))

# Comparar colunas
df = df.withColumn("compareDaysDiff", when(col("days_diff") == col("num_noites"), True).otherwise(False))

# Contar quantas linhas são falsas
false_rows = df.filter(df.compareDaysDiff == False)
num_false_rows = false_rows.count()
num_false_rows

0

In [48]:
# Validar se data de partida é maior que data de chegada
df = df.withColumn("partidaMaiorChegada", when(col("data_chegada") < col("data_partida"), True).otherwise(False))

# Contar falsos
false_rows = df.filter(df.partidaMaiorChegada == False)
num_false_rows = false_rows.count()
num_false_rows

0

In [49]:
# Validar se data de reserva menor que data de chegada 
df = df.withColumn("reservaMenorChegada", when(col("data_reserva") <= col("data_chegada"), True).otherwise(False))

# Contar falsos
false_rows = df.filter(df.reservaMenorChegada == False)
num_false_rows = false_rows.count()
num_false_rows

0

In [50]:
df.write.csv("results")